# Assignment 3
___ 
We'd like to study how __introducing bicycles to a traffic system__ changes the behavior of randomly behaving drivers.
___
We'll model this with _4 flavors_ of cells:
1. Car $C$
2. Bike $B$
3. Empty road $R$
4. Intersection $I$
___
The rules associated with each cell are:
1. Cars can move forward $c$ steps where $c \in [0,k]$ and there are $c$ cells marked $R$ in front of the car.
2. Bikes move forward $b$ steps per timestep if the next $b$ cells are marked $R$.
3. If the cell in front of a car/bike is an $I$ cell, you stall for $s$ timesteps where $s=N_{C}(I) + bN_{B}(I)$ if $b<2$ else $s=N_{C}(I) + N_{B}(I)$ adjacent to $I$ on arrival.
4. At an intersection, $C$ and $B$ choose their next direction with probability $p=\frac{1}{\ell -1}$ where $\ell$ is the number of roads feeding into the intersection.
5. On a two lane road a $C$ may pass a $B$ if there are $2c$ cells empty in the oncoming lane and $2b$ empty cells in front of the $B$.

In [169]:
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib import colors
from matplotlib import rc
#rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
## for Palatino and other serif fonts use:
rc('font',**{'family':'serif','serif':['Palatino']})
rc('text', usetex=True)

import sys, os
import imageio
from glob import glob

In [170]:
type2name = {"I":-1, "E":0, "R":1, "C":2, "B":3}
coin_flip = lambda p: random.random() < p
place = "../figs/traffic_gif/"

In [182]:
def plot_grid(G,ts,fs,es,t):
    plt.close()
    fig = plt.figure(figsize=(10,15))
    gridspec.GridSpec(3,2)
    
    plt.subplot2grid((3,2), (0,0), colspan=2, rowspan=2)
    plt.axis('off')
    bounds = [-1,0,1,2,3,4]
    cmap = colors.ListedColormap(['tomato','honeydew','lightgrey','slateblue','gold'])
    norm = colors.BoundaryNorm(bounds,5)
    plt.imshow(G, interpolation='nearest', cmap=cmap, norm=norm)
    
    plt.subplot2grid((3,2), (2,0), colspan=2, rowspan=1)
    carstats = [f[0] for f in fs]
    bikstats = [f[1] for f in fs]
    avg_cfs = np.mean([cs for cs in carstats if cs >= 0])
    avg_2half_cfs = np.mean([cs for cs in carstats[len(carstats)//3:] if cs >= 0])
    avg_bfs = np.mean([bs for bs in bikstats if bs >= 0])
    avg_exs = np.mean([e for e in es if e >= 0])
    
    plt.plot([ts[0],ts[-1]],[avg_cfs,avg_cfs],color = 'royalblue',alpha=0.6,label='avg. prop. for cars')
    plt.plot([ts[0],ts[-1]],[avg_2half_cfs,avg_2half_cfs],color = 'darkslateblue',alpha=0.6,label='avg. prop. for cars over last third of t interval')
    plt.scatter(ts[:-1], carstats[:-1], s=50,alpha=0.8, facecolors='none', edgecolors='slateblue',label='prop. of cars moving in system at timestep t')
    plt.scatter(ts[-1], carstats[-1], s=50, facecolors='none', edgecolors='navy')
    plt.plot([ts[0],ts[-1]],[avg_bfs,avg_bfs],color = 'khaki',alpha=0.6,label='avg. prop. for bikes')
    plt.scatter(ts[:-1], bikstats[:-1], s=50,alpha=0.8, facecolors='none', edgecolors='gold',label='prop. of bikes moving in system at timestep t')
    plt.scatter(ts[-1], bikstats[-1], s=50, facecolors='none', edgecolors='darkgoldenrod')
    #plt.scatter(ts, es, s=50,alpha=0.4, facecolors='none', edgecolors='crimson',label='out/in ratio for veh in system')
    plt.plot([ts[0],ts[-1]],[avg_exs,avg_exs],color = 'crimson',alpha=0.3,label='avg. out/in ratio for veh in system')
    
    plt.ylim(-0.1,1.1)
    plt.xlabel('timestep, t',fontsize = 10)
    plt.ylabel('propotion of vehicles that moved at timestep, t',fontsize = 10)
    plt.legend(loc=3,fontsize = 8)
    #fig.tight_layout()
    #plt.colorbar(img, cmap=cmap, norm=norm, boundaries=bounds, ticks=bounds)
    plt.savefig(os.path.join(place,"CAgif_timestep_{:04d}".format(t)),bbox_inches='tight')
    #plt.show()
    
    return avg_cfs, avg_2half_cfs, avg_bfs
    
def plot_final_stats(ts,fs,es):    
    plt.close()
    fig = plt.figure(figsize=(5,15))
    
    carstats = [f[0] for f in fs]
    bikstats = [f[1] for f in fs]
    avg_cfs = np.mean([cs for cs in carstats if cs >= 0])
    avg_2half_cfs = np.mean([cs for cs in carstats[len(carstats)//3:] if cs >= 0])
    avg_bfs = np.mean([bs for bs in bikstats if bs >= 0])
    avg_exs = np.mean([e for e in es if e >= 0])
    
    plt.plot([ts[0],ts[-1]],[avg_cfs,avg_cfs],color = 'royalblue',alpha=0.6,label='avg. prop. for cars')
    plt.plot([ts[0],ts[-1]],[avg_2half_cfs,avg_2half_cfs],color = 'darkslateblue',alpha=0.6,label='avg. prop. for cars over last third of t interval')
    plt.scatter(ts[:-1], carstats[:-1], s=50,alpha=0.8, facecolors='none', edgecolors='slateblue',label='prop. of cars moving in system at timestep t')
    plt.scatter(ts[-1], carstats[-1], s=50, facecolors='none', edgecolors='navy')
    plt.plot([ts[0],ts[-1]],[avg_bfs,avg_bfs],color = 'khaki',alpha=0.6,label='avg. prop. for bikes')
    plt.scatter(ts[:-1], bikstats[:-1], s=50,alpha=0.8, facecolors='none', edgecolors='gold',label='prop. of bikes moving in system at timestep t')
    plt.scatter(ts[-1], bikstats[-1], s=50, facecolors='none', edgecolors='darkgoldenrod')
    #plt.scatter(ts, es, s=50,alpha=0.4, facecolors='none', edgecolors='crimson',label='out/in ratio for veh in system')
    plt.plot([ts[0],ts[-1]],[avg_exs,avg_exs],color = 'crimson',alpha=0.3,label='avg. out/in ratio for veh in system')
    
    plt.ylim(-0.1,1.1)
    plt.xlabel('timestep, t',fontsize = 10)
    plt.ylabel('propotion of vehicles that moved at timestep, t',fontsize = 10)
    plt.legend(loc=3,fontsize = 7)
    plt.savefig("../figs/final_stats.pdf",bbox_inches='tight')
    
    
def place_intersections(G):
    """
    - Place intersections on the grid G. 
    - Intersections are placed at any place 4x4 subsection of G of the following forms:
        
        [0][1][1][0]  (1)
        [1][1][1][1]
        [1][1][1][1]
        [0][1][1][0],
    
    The function for now only support intersections of type (1).
    
    args:
        :NxN grid G
    returns:
        :the intersection placed grid
    """
    intersection_regime = np.array([[0,1,1,0],
                               [1,1,1,1],
                               [1,1,1,1],
                               [0,1,1,0]])
    intersection_i, intersection_j = np.where(find_regime(intersection_regime, G) == 1)
    intersections = []
    for i,j in zip(intersection_i, intersection_j):
        G[i+1,j+1] = -1
        G[i+2,j+2] = -1
        G[i+1,j+2] = -1
        G[i+2,j+1] = -1
        intersections.append(Intersection((i,j)))
        #Is = [i+1, i+2, i+1, i+2]
        #Js = [j+1, j+2, j+1, j+2]
        #G[(Is,Js)] = -1
    
    return intersections, G
    

def find_regime(submatrix, grid):
    """
    convolve throughout grid and return masks of indices that describe matches in
    the grid to the specified submatrix
    
    args:
        :submatrix -   2d array of dimensions n x m 
        :grid      -   2d array symbolizing the world of dimensions N x M
    returns:
        :masks that describe matches of the regime in the grid
    """
    n, m = submatrix.shape
    N, M = grid.shape
    convolve_mat = np.zeros((N-n+1, M-m+1))
    
    for row in range(N - n + 1):
        for col in range(M - m + 1):
            view = grid[row:row + n, col:col + m]
            if np.array_equal(view, submatrix):
                convolve_mat[row, col] = 1
            else:
                convolve_mat[row, col] = 0
                
    return convolve_mat    

# instantiate a grid
# find locations of intersections
# for each loc:
#    - generate an Intersection object at that location
#    - record that location
class Intersection:
    def __init__(self, coordinates):
        i, j = self.i, self.j = coordinates
        
        # compute the neighborhood of coordinates
        # for this intersection
        self.nsew = {"N":(i,   j+1),
                     "S":(i+3, j+2),
                     "E":(i+1, j+3),
                     "W":(i+2, j)}
        
        self.potential_locs = {"N":[(i+1,j),
                                    (i+3,j+1),
                                    (i+2,j+3)],
                               "S":[(i+1,j),
                                    (i+2,j+3),
                                    (i,  j+2)],
                               "E":[(i+1,j),
                                    (i,  j+2),
                                    (i+3,j+1)],
                               "W":[(i,  j+2),
                                    (i+2,j+3),
                                    (i+3,j+1)]}
        self.queue = []
    
    
    
    def __str__(self):
        return "Intersection @ ({},{})".format(self.i,self.j)
    
    def __repr__(self):
        return str(self)
    
    
    
class Passing_regime:
    def __init__(self, direction, coordinates):
        i, j = self.i, self.j = coordinates
        
        if direction == 'Ebound':
            self.regime = np.array([[1,1,1,1],
                                  [2,3,1,1]])
            self.old_loc = (i+1, j)
            
            self.new_loc = (i+1, j+3)
            
        elif direction == 'Wbound':
            self.regime = np.array([[1,1,3,2],
                                  [1,1,1,1]])
            self.old_loc = (i,   j+3)
            
            self.new_loc = (i,   j)
            
        elif direction == 'Nbound':
            self.regime = np.array([[1,1],
                                  [1,1],
                                  [1,3],
                                  [1,2]])
            self.old_loc = (i+3,  j+1)
            
            self.new_loc = (i,   j+1)
            
        elif direction == 'Sbound':
            self.regime = np.array([[2,1],
                                  [3,1],
                                  [1,1],
                                  [1,1]])
            self.old_loc = (i,   j)
            
            self.new_loc = (i+3, j)
            

    def __str__(self):
        return "Car can pass bike @ ({},{})".format(self.old_loc[0],self.old_loc[1])
    
    def __repr__(self):
        return str(self)
    
        
    
def lay_roads(N_traffic, W_traffic, G):
    """
    args:
        N_traffic: (X length list) of x coords to roads
        W_traffic: (Y length list) of y coords to roads
    returns:
        :(dict) maps from direction in {N,S,E,W} to 
         traffic incoming coordinates from N,S,E,W respectively
    """
    x,y = G.shape
    S_traffic = [i+1 for i in N_traffic]
    E_traffic = [j-1 for j in W_traffic]
    
    # laying roads
    for i1,i2 in zip(N_traffic,S_traffic):
        G[:,i1] = 1
        G[:,i2] = 1
        
    for j1,j2 in zip(W_traffic,E_traffic):
        G[j1,:] = 1
        G[j2,:] = 1    
    
    return {"N":[(0, j) for j in N_traffic],"W":[(i, 0) for i in W_traffic],
            "S":[(x-1,j) for j in S_traffic],"E":[(i,y-1) for i in E_traffic]}

def move_vehicle(vtype, vi, vj, shadow, G):
    M, N = G.shape
    northmost = (vi == 0 and shadow[vi, vj+1] == type2name["E"])
    southmost = (vi == N - 1 and shadow[vi, vj-1] == type2name["E"])
    eastmost = (vj == M - 1 and shadow[vi+1, vj] == type2name["E"])
    westmost = (vj == 0 and shadow[vi - 1, vj] == type2name["E"])
    if northmost or southmost or eastmost or westmost:
        G[vi,vj] = type2name["R"]
        return {'veh_move':True, 'veh_exit':True}
    
    # E V 
    #   R
    try:
        if shadow[vi+1, vj] == type2name["R"] and shadow[vi, vj -1] == type2name["E"]:
            G[vi,vj] = type2name["R"]
            G[vi+1, vj] = type2name[vtype]
            return {'veh_move':True, 'veh_exit':False}
    except IndexError:
        pass
    
    # R
    # V E
    try:
        if shadow[vi-1, vj] == type2name["R"] and shadow[vi, vj+1] == type2name["E"]:
            G[vi,vj] = type2name["R"]
            G[vi-1, vj] = type2name[vtype]
            return {'veh_move':True, 'veh_exit':False}
    except IndexError:
        pass  
    # V R
    # E
    try:
        if shadow[vi,vj+1] == type2name["R"] and shadow[vi+1,vj] == type2name["E"]:
            G[vi,vj] = type2name["R"]
            G[vi, vj+1] = type2name[vtype]
            return {'veh_move':True, 'veh_exit':False}
    except IndexError:
        pass
    # E
    # R V
    try:
        if shadow[vi,vj-1] == type2name["R"] and shadow[vi-1, vj] == type2name["E"]:
            G[vi,vj] = type2name["R"]
            G[vi, vj-1] = type2name[vtype]
            return {'veh_move':True, 'veh_exit':False}
    except IndexError:
        pass

    
def cars_pass_bikes(shadow,G):
    ''' 
    perform a regime search for cars that could pass bicycles
    args: shadow: current state of the system
          G     : next state of the system

    Bike Passing Regimes:

     R R R R  eastbound  -> move C from (i+1, j  ) to (i+1, j+3)
     C B R R 

     R R B C  westbound  -> move C from (i  , j+3) to (i  , j  )
     R R R R

     R R      northbound -> move C from (i+3, j+1) to (i  , j+1)
     R R
     R B
     R C

     C R      southbound -> move C from (i  , j  ) to (i+3, j  )
     B R
     R R
     R R
    '''
    
    bikepass_regimes = {'Ebound':np.array([[1,1,1,1],
                                  [2,3,1,1]]),
                        'Wbound':np.array([[1,1,3,2],
                                  [1,1,1,1]]),
                        'Nbound':np.array([[1,1],
                                  [1,1],
                                  [1,3],
                                  [1,2]]),
                        'Sbound':np.array([[2,1],
                                  [3,1],
                                  [1,1],
                                  [1,1]]),}

    carmoves = 0
    for direction in bikepass_regimes:
        regimei,regimej = np.where(find_regime(bikepass_regimes[direction],shadow) == 1)
        for i,j, in zip(regimei,regimej):
            R = Passing_regime(direction,(i,j))
            G[R.old_loc] = type2name['R']
            G[R.new_loc] = type2name['C']
            carmoves += 1
    
    return carmoves

In [183]:
os.system("rm {}".format(os.path.join(place,"*")))
plt.close()
bike2car  = 0.2  # proportion of vehicles that enter the system that are bikes
b_rate = 3       # timestep (speed differential) of bikes (bikes move once every b_rate t)
dim = 100        # dimensions of the system/grid
time = 500      # total time system runs for
system = np.zeros((dim,dim)) 

# flow_rates: (dict) - probability of vehicles entering the system
#                     from N,S,E, or W directions @ time t
allflowseq = True
if allflowseq:
    flowrate = 0.4
    flows = {d:flowrate for d in set("NSEW")}
else:
    flows = {"N":0.2,
             "S":0.1,
             "E":0.05,
             "W":0.15}

# incoming traffic zones:|
N_traffic = [dim//4,dim//2,dim - dim//4]
W_traffic = [dim//4,dim//2,dim - dim//4]

# laying roads
traffic_in    = lay_roads(N_traffic,W_traffic,system)
intersections, system = place_intersections(system)

flowstats = [] # number of vehicles that move per time step / total vehicles in system
exitflows = []
timesteps = [] 

for t in range(time):
    shadow = system.copy()
    carmove = 0
    bikemove = 0
    # vehicles entering the system
    veh_in = 0
    veh_out = 0
    for direction in flows:
        for t_in in traffic_in[direction]:
            if shadow[t_in] == type2name["R"] and coin_flip(flows[direction]):
                if coin_flip(bike2car):
                    system[t_in] = type2name["B"]   # a bike appears
                else:
                    system[t_in] = type2name["C"]   # a car appears
                veh_in +=1    
                
    # do general, nonintersection related movement
    if t % b_rate == 0:
        # find and move bikes
        for bi, bj in zip(*np.where(shadow == type2name["B"])):
            mv = move_vehicle("B",bi, bj, shadow, system)
            if mv:
                if mv['veh_move']:
                    bikemove +=1
                    if mv['veh_exit']:
                        veh_out +=1
    
    for ci, cj in zip(*np.where(shadow == type2name["C"])):
        mv = move_vehicle("C",ci,cj,shadow,system)
        if mv:
            if mv['veh_move']:
                carmove +=1
                if mv['veh_exit']:
                    veh_out +=1
    
    # look at intersections, move appropriately
    # who's at an intersection?
    for I in intersections:
        for direction in I.nsew.keys():
            if shadow[I.nsew[direction]] > 1 and direction not in I.queue:
                I.queue.append(direction)
        
        if I.queue:
            dmover = I.queue.pop(0)
            i,j = I.nsew[dmover]
            next_coord = random.choice(I.potential_locs[dmover])
            if shadow[next_coord] == type2name["R"]:
                system[next_coord] = shadow[i,j]
                system[i,j] = type2name["R"]
                if shadow[i,j] == 'C':
                    carmove +=1
                elif shadow[i,j] =='B':
                    bikemove +=1
    
    carmove += cars_pass_bikes(shadow,system)
    
    # total number of C and B at start of timestep:
    totcar = np.count_nonzero(shadow == type2name['C']) 
    totbike = np.count_nonzero(shadow == type2name['B'])

    if totcar == 0:
        carstat = -1
    else:
        carstat = carmove/totcar #+ 0.01
    
    if veh_in == 0:
        existat = -1
    else:
        existat = veh_out/veh_in
        
    if totbike == 0:
        bikestat = -1
    else:
        if t % b_rate == 0:
            bikestat = bikemove/totbike #- 0.01
        else:
            bikestat = -1 #flowstats[-1][1]
    flowstats.append((carstat,bikestat))
    exitflows.append(existat)
    timesteps.append(t)
    
    avg_cfs, avg_2half_cfs , avg_bfs= plot_grid(system,timesteps,flowstats,exitflows,t)
    
    if t == time - 1:
        plot_final_stats(timesteps,flowstats,exitflows)

/Applications/anaconda/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  Parameters
/Applications/anaconda/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  


In [184]:
# Get all gif image files
imgfiles = sorted(glob(place+'*'))

# Make GIF!!!!   Naming convention: 
#                                t: number of timesteps
#                               br: bikerate (bike speed delay)
#                              b2c: proportion of bikes in the system
#                            flows: single flow rate if uniform from every direction
#                                   or flow for each direction 
#                   avg cflow stat: average proportion of cars moving per time step over entire time interval
#                                   average proportion of cars moving per time step over last 1/3 of time interval
#                   avg bflow stat: average proportion of bikes moving per time step

if allflowseq:
    path = '_t-{}_br-{}_b2c-{}_flows-{}_avgcf-{}-{}_avgbf-{}'.format(time,b_rate,round(bike2car,2),round(flows['N'],2),round(avg_cfs,3),round(avg_2half_cfs,3),round(avg_bfs,3))
else:
    path = '_t-{}_br-{}_b2c-{}_flowN-{}-S-{}-E-{}-W-{}_avgcf-{}-{}_avgbf-{}'.format(time,b_rate,round(bike2car,2),round(flows['N'],2),round(flows['S'],2),round(flows['E'],2),round(flows['W'],2),round(avg_cfs,3),round(avg_2half_cfs,3),round(avg_bfs,3))
with imageio.get_writer('../figs/traffic_gif'+path+'.gif', mode='I') as writer:
    for filename in imgfiles:
        image = imageio.imread(filename)
        writer.append_data(image)

os.system('scp ../figs/final_stats.pdf ../figs/final_stats'+path+'.pdf')

0